In [1]:
import sys, inspect
import cobra
#If you have CPLEX, uncomment this
import cplex
import cobrakbase
#Put the path to ModelSEEDpy on your machine here
sys.path.append("/Users/chenry/code/ModelSEEDpy")
from modelseedpy import MSPackageManager, FBAHelper, MSBuilder

ModuleNotFoundError: No module named 'cobrakbase'

In [ ]:
kbase_api = cobrakbase.KBaseAPI()
model = kbase_api.get_from_ws("Hot_Lake_seven.mdl",93544)
#If you have CPLEX, uncomment this
model.solver = 'optlang-cplex'
template = kbase_api.get_from_ws("GramNegModelTemplateV3","NewKBaseModelTemplates")
media = kbase_api.get_from_ws("HotLakeMedia",93544)
model = MSBuilder.gapfill_model(model,"bio1",template,None)
#Getting package manager to create and track community model packages
FBAHelper.set_objective_from_target_reaction(model,"bio1")
pkgmgr = MSPackageManager.get_pkg_mgr(model)
pkgmgr.getpkg("KBaseMediaPkg").build_package(None)
pkgmgr.getpkg("CommKineticPkg").build_package(2100,None,True)
with open('CommunityFBA.lp', 'w') as out:
    out.write(str(model.solver))
for i in range(1,8,1):
    FBAHelper.set_objective_from_target_reaction(model,"DM_cpd11416_c"+str(i))
    sol=model.optimize()
    print(str(i)+":"+str(sol.objective_value))